In [91]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim


In [55]:
movie_ratings = pd.read_csv('./dataset/ratings.csv')
movie_ratings_small = pd.read_csv('./dataset/ratings_small.csv')
credits = pd.read_csv('./dataset/credits.csv')
links = pd.read_csv('./dataset/links.csv')
links_small = pd.read_csv('./dataset/links_small.csv')
movies_metadata = pd.read_csv('./dataset/movies_metadata.csv')
keywords = pd.read_csv('./dataset/keywords.csv')

/var/folders/s7/_6xph2kx7zj1t3c3tmb0gk5c0000gn/T/ipykernel_6193/983951038.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv('./dataset/movies_metadata.csv')


In [56]:
movie_ratings.isnull().sum()
#movies_metadata.head()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [57]:
movies_metadata.drop_duplicates(subset='id',keep='first', inplace=True)

In [58]:
movies_metadata = movies_metadata[movies_metadata['id'].str.isdigit()] 
movies_metadata['id'] = movies_metadata['id'].astype('int64')

In [74]:

#movies_metadata = movies_metadata[movies_metadata['id'].str.isdigit()] 
movies_metadata['userId'] = movies_metadata['userId'].astype('int64')

In [60]:
movies_metadata = movies_metadata.merge(movie_ratings, left_on='id', right_on='movieId', how='left')

In [66]:
movies_metadata.dropna(subset='userId', inplace= True)
movies_metadata.drop(columns=['movieId'], inplace=True)

In [67]:
movies_metadata[['id','userId','rating']]

,id,userId,rating
0,862,1923.0,3.0
1,862,2103.0,5.0
2,862,5380.0,1.0
3,862,6177.0,4.0
4,862,6525.0,4.0
...,...,...,...
11474428,111109,33940.0,2.5
11474429,111109,172224.0,3.0
11474430,111109,210792.0,3.0
11474431,111109,225396.0,3.5


In [75]:
movies_metadata['userId'].dtype

dtype('int64')

In [76]:
# Neural MF Model
class NeuralMF(nn.Module):
    def __init__(self, num_users, num_items, latent_dim):
        super(NeuralMF, self).__init__()

        # Embeddings
        self.user_embedding = nn.Embedding(num_users, latent_dim)
        self.item_embedding = nn.Embedding(num_items, latent_dim)

        # Neural network layers
        self.fc1 = nn.Linear(latent_dim * 2, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

        self.activation = nn.ReLU()

    def forward(self, user, item):
        user_embed = self.user_embedding(user)
        item_embed = self.item_embedding(item)

        # Concatenate user and item embeddings
        x = torch.cat([user_embed, item_embed], dim=-1)

        # Feed through fully connected layers
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        return x.squeeze()

In [114]:
latent_dim = 16
batch_size = 4
epochs = 10
learning_rate = 0.001


movies_metadata['user_idx'] = movies_metadata['userId'].astype('category').cat.codes
movies_metadata['item_idx'] = movies_metadata['id'].astype('category').cat.codes

# 유저 및 아이템 텐서, 평점 텐서
user_ids = torch.tensor(movies_metadata['user_idx'].values, dtype=torch.long)
item_ids = torch.tensor(movies_metadata['item_idx'].values, dtype=torch.long)
ratings = torch.tensor(movies_metadata['rating'].values, dtype=torch.float32)

#user_ids
num_users = movies_metadata['user_idx'].nunique()
num_items = movies_metadata['item_idx'].nunique()
model = NeuralMF(num_users, num_items, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    
    # 예측값 계산
    predicted_ratings = model(user_ids, item_ids)
    
    # 손실 계산
    loss = criterion(predicted_ratings, ratings)
    
    # 역전파 및 최적화
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 출력
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 예측된 평점 출력
model.eval()
predicted_ratings = model(user_ids, item_ids)
print("Predicted Ratings:", predicted_ratings)



Epoch [10/100], Loss: 10.2024
Epoch [20/100], Loss: 5.6569
Epoch [30/100], Loss: 2.1127
Epoch [40/100], Loss: 1.6438
Epoch [50/100], Loss: 1.5607
Epoch [60/100], Loss: 1.3991
Epoch [70/100], Loss: 1.3909
Epoch [80/100], Loss: 1.3451
Epoch [90/100], Loss: 1.3239
Epoch [100/100], Loss: 1.2994
Predicted Ratings: tensor([3.8541, 3.8256, 4.0133,  ..., 3.5408, 3.1712, 2.3865],
       grad_fn=<SqueezeBackward0>)


In [113]:
user_ids

tensor([  1882,   2061,   5284,  ..., 206928, 221262, 264653])

In [106]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# 예제 데이터셋 (유저 ID, 아이템 ID, 실제 평점)
data = pd.DataFrame({
    'user_id': ['u1', 'u2', 'u3', 'u1', 'u2'],
    'item_id': ['i1', 'i2', 'i3', 'i1', 'i3'],
    'rating': [5.0, 4.0, 3.0, 4.5, 2.0]
})

# 유저 및 아이템 ID를 정수 인덱스로 변환
data['user_idx'] = data['user_id'].astype('category').cat.codes
data['item_idx'] = data['item_id'].astype('category').cat.codes

# 유저 및 아이템 텐서, 평점 텐서
user_ids = torch.tensor(data['user_idx'].values, dtype=torch.long)
item_ids = torch.tensor(data['item_idx'].values, dtype=torch.long)
ratings = torch.tensor(data['rating'].values, dtype=torch.float32)

print("user ids", user_ids)
# 유저 및 아이템 임베딩 레이어 정의
# num_users = len(torch.unique(user_ids))
# num_items = len(torch.unique(item_ids))
# embedding_dim = 8

# user_embedding = nn.Embedding(num_users, embedding_dim)
# item_embedding = nn.Embedding(num_items, embedding_dim)

# # 예측 모델 정의
# class MatrixFactorization(nn.Module):
#     def __init__(self, num_users, num_items, embedding_dim):
#         super(MatrixFactorization, self).__init__()
#         self.user_embedding = nn.Embedding(num_users, embedding_dim)
#         self.item_embedding = nn.Embedding(num_items, embedding_dim)
    
#     def forward(self, user_ids, item_ids):
#         user_vecs = self.user_embedding(user_ids)
#         item_vecs = self.item_embedding(item_ids)
#         return (user_vecs * item_vecs).sum(dim=1)  # 유저와 아이템 벡터의 내적을 예측값으로 사용

# # 모델 초기화
# model = MatrixFactorization(num_users, num_items, embedding_dim)

# # 손실 함수 (MSE) 및 옵티마이저 (Adam)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)

# # 훈련 과정
# num_epochs = 100
# for epoch in range(num_epochs):
#     model.train()
    
#     # 예측값 계산
#     predicted_ratings = model(user_ids, item_ids)
    
#     # 손실 계산
#     loss = criterion(predicted_ratings, ratings)
    
#     # 역전파 및 최적화
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     # 출력
#     if (epoch + 1) % 10 == 0:
#         print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # 예측된 평점 출력
# model.eval()
# predicted_ratings = model(user_ids, item_ids)
# print("Predicted Ratings:", predicted_ratings)


user ids tensor([0, 1, 2, 0, 1])


In [99]:
model = NeuralMF()

(tensor([[ 0.9074,  0.6172, -1.9166,  ...,  2.1840,  1.1550, -1.9553],
         [-0.0825, -1.0231, -1.1063,  ...,  0.2507,  0.7501,  1.2624],
         [ 0.1195,  0.4591,  0.3297,  ...,  0.7087, -1.7993,  0.5238],
         ...,
         [-0.3801, -0.5860, -0.9542,  ...,  0.1646, -0.6535, -2.5811],
         [ 0.2262, -0.5687,  0.2600,  ..., -0.3639, -2.4057, -1.4420],
         [-0.4908,  1.6719,  0.9135,  ..., -0.0377,  1.4994, -2.1600]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[ 0.8322, -1.5359,  0.1454,  ..., -0.8599, -1.5215,  0.3334],
         [ 0.8322, -1.5359,  0.1454,  ..., -0.8599, -1.5215,  0.3334],
         [ 0.8322, -1.5359,  0.1454,  ..., -0.8599, -1.5215,  0.3334],
         ...,
         [-1.1115, -0.8108,  0.3024,  ..., -0.5019,  1.7716,  0.7275],
         [-1.1115, -0.8108,  0.3024,  ..., -0.5019,  1.7716,  0.7275],
         [-1.1115, -0.8108,  0.3024,  ..., -0.5019,  1.7716,  0.7275]],
        grad_fn=<EmbeddingBackward0>))